In [ ]:
import MeCab
import pandas as pd
import numpy as np
import mysql.connector as mydb
import pandas.io.sql as psql
import collections

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
def devide_by_mecab(text):
    tagger = MeCab.Tagger("-Ochasen")
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        if pos in ["名詞"]:
            word = node.surface
            word_list.append(word)
        node = node.next
    return "  ".join(word_list)

In [ ]:
connection = mydb.connect(
  host = 'localhost',
  port = '3306',
  user = 'username',
  password = 'password',
  database = 'dbname'
)

In [ ]:
df_frontend = psql.read_sql("SELECT * FROM contents WHERE search_word = 'フロントエンド'",connection)
df_serverside = psql.read_sql("SELECT * FROM contents WHERE search_word = 'サーバーサイド'",connection)
df_game = psql.read_sql("SELECT * FROM contents WHERE search_word = 'ゲームプログラマー'",connection)

In [ ]:
df_frontend.info()
df_serverside.info()
df_game.info()

In [ ]:
# need_skillsを名詞に分解して返すメソッド
def get_all_words(df):
    all_words = []
    for index, row in df.iterrows():
        words = devide_by_mecab(row['need_skills']).split()
        all_words.extend(words)
    return all_words

In [ ]:
# 出現頻度順に出力するメソッド
def print_count_of_words(all_words):
    word_counts = collections.Counter(all_words).most_common()
    for word_count in word_counts:
        print(word_count[0],word_count[1])

In [ ]:
count_of_words_frontend = collections.Counter(get_all_words(df_frontend))
count_of_words_serverside = collections.Counter(get_all_words(df_serverside))
count_of_words_game = collections.Counter(get_all_words(df_game))

In [ ]:
count_of_words_frontend.most_common()

In [ ]:
count_of_words_serverside.most_common()

In [ ]:
count_of_words_game.most_common()

In [ ]:
# スキルに関係する単語,count >= 6
frontend_top_words = ['JavaScript', 'CSS', 'HTML', 'js', 'Vue', 'React', 'デザイン', 'チーム', 'Git', 'UI', 'PHP', 'UX', 'Angular', 'Ruby', 'Javascript', 'jQuery', 'Photoshop', 'コミュニケーション', 'API', 'TypeScript', 'SPA', 'Java', 'Sass', 'デザイナー', 'Illustrator', 'JS', 'テスト', 'サーバー', 'webpack', 'GitHub', 'AWS', 'AngularJS', 'WordPress', 'Webpack', 'Rails', 'iOS', 'CMS', 'Python', 'Redux', 'MySQL', 'Gulp', 'Android', 'gulp', 'C', 'SCSS', 'git', 'DB', 'Linux', 'Babel', 'Docker', 'CI']
serverside_top_words = ['Ruby', 'PHP', 'AWS', 'Python', 'C', 'Java', 'サーバー', 'Rails', 'チーム', 'インフラ', 'js', 'Git', 'サーバ', 'Android', 'JavaScript', 'Go', 'Linux', 'Perl', 'HTML', 'MySQL', 'RDBMS', 'CSS', 'クラ', 'ウド', 'API', 'フロント', 'マネジメント', 'GitHub', 'iOS', 'DB', 'GCP', 'React', 'Vue', 'ネットワーク', 'Node', 'HTTP', 'Swift', 'CI', 'Objective', 'Docker', 'セキュリティ', 'Javascript', 'Azure', 'ネイティブ', 'PostgreSQL', 'アーキテクチャ', 'SQL', 'テスト', '#', 'スマート', 'フォン', 'UI', 'MVC', 'コミュニケーション', 'git', 'Scala', 'Kotlin' , 'CD', 'データベース', 'TypeScript', 'Apache', 'LAMP', 'デザイナー', 'コンテナ', 'RDB', 'Laravel']
game_top_words = ['C', '3', 'D', 'Unity', 'Java', 'PHP', 'デザイン', '++', 'サーバー', '++、', 'ネットワーク', 'JavaScript', 'Android', 'マネジメント', '#、', 'Objective', 'Photoshop', 'Maya', 'チーム', 'デザイナー', 'Linux', 'MySQL', 'Ruby', 'Python', 'インフラ', '#', 'グラフィックス', 'サーバ', 'Excel', 'グラフィック', 'コミュニケーション', 'Unreal', 'DCG', 'AWS', 'Perl', 'Illustrator', 'Engine', 'プランナー', 'Word', 'ネイティブ', 'モーション', 'ディレクター', 'HTML', 'UI', 'Flash', 'エフェクト', 'VB', 'サウンド', 'DS', 'OpenGL', 'iOS', 'DirectX']

In [ ]:
def get_top_word(top_words,count_of_words):
    df = pd.DataFrame({})
    for i,word in enumerate(top_words):
        word_data = pd.Series([word,count_of_words[word]], index=['word','count'], name=i)
        df = df.append(word_data)
    return df

In [ ]:
df_frontend_top_words =  get_top_word(frontend_top_words,count_of_words_frontend)
df_serverside_top_words =  get_top_word(serverside_top_words,count_of_words_serverside)
df_game_top_words =  get_top_word(game_top_words,count_of_words_game)

In [ ]:
for df in [df_frontend_top_words,df_serverside_top_words,df_game_top_words]:
    df['rank'] = df['count'].rank(ascending = False, method = 'min').astype(int)
    df['count'] = df['count'].astype(int)

In [ ]:
df_frontend_top_words[['rank','word','count']]

In [ ]:
df_serverside_top_words[['rank','word','count']]

In [ ]:
df_game_top_words[['rank','word','count']]